In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import yfinance as yf

from datetime import date

Crear cartera equiponderada (sin rebalanceos) compuesta por: 
AAPL, AMZN, BAC, BRK-B, CVX, ENPH, GLD, GME, GOOGL, JNJ, JPM, MSFT, NVDA, PG, XOM, bono del gobierno americano a 10 años, bono del gobierno americano a 2 años y HYG (BAMLH0A0HYM2). 

Fase 1: Detectando el "Pulso" del Mercado (Hidden Markov Models) 
Objetivo: Considerando la existencia de dos estados de mercado (calma y crisis), identifica, 
para cada día, en qué estado se encuentra el mercado. 
• Tarea  Técnica:  Ajustar  un  modelo  Gaussian  HMM  (Hidden  Markov  Model)  de  2 
estados sobre la serie de retornos multivariante y/o sobre factores representativos. 
• Output Visual: Gráfico del S&P 500 coloreado según el régimen detectado (Blanco = 
Calma, Azul = Crisis).

In [2]:
date_init = "2006-01-01"
symbols = ['AAPL','AMZN','BAC','BRK-B','CVX','ENPH','GLD','GME','GOOGL','JNJ','JPM','MSFT','NVDA','PG','XOM']
bons = ['DGS10', 'DGS2']
hyg = 'BAMLH0A0HYM2'

In [36]:
# descargar precios ajustados desde Yahoo hasta hoy
end = pd.Timestamp.today().normalize()
prices = yf.download(symbols, start=date_init, end = end, auto_adjust=True, progress=False)
if isinstance(prices, pd.Series):  # si solo hay un ticker
    prices = prices.to_frame()

prices = prices.rename(columns=str.lower)
prices.index.name = 'date'
# calcular retornos diarios
returns = prices.pct_change().dropna()
returns = returns.reset_index() 
returns.head()

Price        date     close                                                    \
Ticker                 aapl      amzn       bac     brk-b       cvx      enph   
0      2012-04-02  0.031824 -0.022024  0.011494  0.008503  0.010167  0.064033   
1      2012-04-03  0.017280  0.008129 -0.019628 -0.003666 -0.010711 -0.021767   
2      2012-04-04 -0.007961 -0.028398 -0.030559 -0.006009 -0.014374 -0.024869   
3      2012-04-05  0.015008  0.002062  0.003261 -0.001727 -0.008049 -0.002685   
4      2012-04-09  0.004024 -0.012964 -0.032503 -0.014213 -0.012029 -0.060565   

Price                                 ...    volume                      \
Ticker       gld       gme     googl  ...      enph       gld       gme   
0       0.005058  0.028388  0.008858  ... -0.901854  0.001022  0.001742   
1      -0.018719 -0.018700 -0.006647  ... -0.361049  1.230571 -0.214851   
2      -0.016761 -0.004083 -0.011624  ... -0.478522 -0.050332 -0.274077   
3       0.006997 -0.010023 -0.004456  ... -0.368262 -0.438663 -0.014764   
4       0.006696  0.005523 -0.002341  ...  0.182167 -0.080500 -0.100520   

Price                                                                         
Ticker     googl       jnj       jpm      msft      nvda        pg       xom  
0      -0.011468 -0.223016  0.066427  0.129269 -0.226584 -0.008662  0.068243  
1      -0.104763  0.038983  0.121691  0.192407  0.055668 -0.190932  0.278749  
2      -0.204069  0.033292  0.332078  0.156806  0.151971  0.203923 -0.033988  
3       0.424613 -0.160046 -0.302606  0.018455 -0.255037 -0.080731 -0.301418  
4      -0.058654 -0.196814  0.018871 -0.383417 -0.010611 -0.104713 -0.077601  

[5 rows x 76 columns]

In [24]:
BASE_URL = "https://fred.stlouisfed.org/graph/fredgraph.csv?id="

def download_fred(serie):
    """
    Descarga cualquier serie de FRED usando solo pandas.
    serie: código de la serie en FRED (ej: 'DGS10', 'DGS2', 'VIXCLS')
    """
    url = BASE_URL + serie
    df = pd.read_csv(url)

    # FRED usa 'observation_date' como nombre de columna cambio por 'date' y convertir a datetime
    df.rename(columns={"observation_date": "date"}, inplace=True)
    df["date"] = pd.to_datetime(df["date"])

    # Filtrar por fecha de inicio date_init
    df = df[df["date"] >= pd.to_datetime(date_init)]


    # Calcular cambio diario
    df[f"{serie}_change"] = df[serie].diff()

    return df


In [25]:
vixcls = download_fred("VIXCLS")
print(vixcls.head())


           date  VIXCLS  VIXCLS_change
4174 2006-01-02     NaN            NaN
4175 2006-01-03   11.14            NaN
4176 2006-01-04   11.37           0.23
4177 2006-01-05   11.31          -0.06
4178 2006-01-06   11.00          -0.31


In [26]:
dgs10 = download_fred("DGS10")
print(dgs10.head())


            date  DGS10  DGS10_change
11479 2006-01-02    NaN           NaN
11480 2006-01-03   4.37           NaN
11481 2006-01-04   4.36         -0.01
11482 2006-01-05   4.36          0.00
11483 2006-01-06   4.38          0.02


In [27]:
dgs2 = download_fred("DGS2")
print(dgs2.head())

           date  DGS2  DGS2_change
7719 2006-01-02   NaN          NaN
7720 2006-01-03  4.34          NaN
7721 2006-01-04  4.31        -0.03
7722 2006-01-05  4.32         0.01
7723 2006-01-06  4.36         0.04


In [28]:
hy_oas = download_fred("BAMLH0A0HYM2")

print(hy_oas.head())


           date  BAMLH0A0HYM2  BAMLH0A0HYM2_change
2380 2006-01-02           NaN                  NaN
2381 2006-01-03          3.73                  NaN
2382 2006-01-04          3.69                -0.04
2383 2006-01-05          3.64                -0.05
2384 2006-01-06          3.56                -0.08


In [37]:
df_final = (
    returns
    .merge(vixcls[["date", "VIXCLS_change"]], on="date", how="left")
    .merge(dgs10[["date", "DGS10_change"]], on="date", how="left")
    .merge(dgs2[["date", "DGS2_change"]], on="date", how="left")
    .merge(hy_oas[["date", "BAMLH0A0HYM2_change"]], on="date", how="left")
)


print(df_final.head())


MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)